In [1]:
import tkinter as tk
from tkinter import filedialog
from tkinter import simpledialog
from PIL import Image
import numpy as np
import hashlib

selected_img_path = None
password_hash = None


def generate_hash(password):
    global password_hash
    hash_object = hashlib.sha256(password.encode())
    password_hash = hash_object.hexdigest()

def encode_and_save(img_path, message, save_path):
    global password_hash

    if not message:
        output_label.configure(text="Please enter a message!")
        return

    password = password_entry.get()
    if password:
        generate_hash(password)
    else:
        output_label.configure(text="Please enter a password!")
        return

    img = Image.open(img_path)
    img_array = np.array(img)

    binary_message = bin(len(message))[2:].zfill(64) + ''.join(format(ord(char), '08b') for char in message)

    message_index = 0
    for i in range(len(img_array)):
        for j in range(len(img_array[0])):
            for k in range(3):  # Handling RGB channels
                if message_index < len(binary_message):
                    img_array[i][j][k] = (img_array[i][j][k] & ~1) | int(binary_message[message_index])
                    message_index += 1

    encoded_image = Image.fromarray(img_array.astype(np.uint8))
    encoded_image.save(save_path)
    output_label.configure(text="Message encoded and saved successfully!")

# Previous code...

def decode():
    global password_hash

    encoded_img_path = filedialog.askopenfilename()
    if not encoded_img_path:
        return

    password = None
    if password_hash is not None:
        password = simpledialog.askstring("Password", "Enter password:", show='*')
        if not password:
            return

    if password is not None and hashlib.sha256(password.encode()).hexdigest() != password_hash:
        output_text.configure(text="Invalid password!")
        return

    img = Image.open(encoded_img_path)
    img_array = np.array(img)

    extracted_bits = []
    end_indicator = [1, 1, 1, 1, 1, 1, 1, 0]  # Indicator for the end of the message
    end_len = len(end_indicator)
    message_found = False

    for i in range(len(img_array)):
        for j in range(len(img_array[0])):
            for k in range(3):  # Handling RGB channels
                extracted_bit = img_array[i][j][k] & 1
                extracted_bits.append(extracted_bit)

                if len(extracted_bits) >= end_len and extracted_bits[-end_len:] == end_indicator:
                    message_found = True
                    extracted_bits = extracted_bits[:-end_len]
                    break
            if message_found:
                break
        if message_found:
            break

    if message_found:
        extracted_message = ''.join(str(bit) for bit in extracted_bits)

        message_length = int(extracted_message[:64], 2)
        message = extracted_message[64:64 + message_length * 8]

        decoded_message = ""
        for l in range(0, len(message), 8):
            byte = message[l:l + 8]
            decoded_message += chr(int(byte, 2))

        output_text.configure(text="Decoded Message: " + decoded_message)
    else:
        output_text.configure(text="Message not found or invalid password!")



def select_image():
    global selected_img_path
    selected_img_path = filedialog.askopenfilename()



def create_gui():
    global password_entry, output_label, output_text

    root = tk.Tk()
    root.title("Steganography Encoder/Decoder")

    # Creating frames for a better layout
    top_frame = tk.Frame(root)
    top_frame.pack()

    middle_frame = tk.Frame(root)
    middle_frame.pack()

    bottom_frame = tk.Frame(root)
    bottom_frame.pack()

    # Select Image section
    select_image_label = tk.Label(top_frame, text="Select Image:")
    select_image_label.pack(side=tk.LEFT)

    select_image_button = tk.Button(top_frame, text="Browse", command=select_image)
    select_image_button.pack(side=tk.LEFT)

    # Rest of the code...


    # Enter Message section
    message_label = tk.Label(middle_frame, text="Enter message to hide:")
    message_label.pack()

    message_entry = tk.Entry(middle_frame, width=50)
    message_entry.pack()

    # Enter Password section
    password_label = tk.Label(middle_frame, text="Enter password:")
    password_label.pack()

    password_entry = tk.Entry(middle_frame, width=50, show="*")
    password_entry.pack()

    # Buttons
    encode_button = tk.Button(bottom_frame, text="Encode & Save", command=lambda: encode_and_save(selected_img_path, message_entry.get(), "encoded_image.png"))
    encode_button.pack(side=tk.LEFT)

    decode_button = tk.Button(bottom_frame, text="Decode", command=decode)
    decode_button.pack(side=tk.LEFT)

    # Output Labels
    output_label = tk.Label(root, text="", fg="red")
    output_label.pack()

    output_text = tk.Label(root, text="", fg="blue")
    output_text.pack()

    root.mainloop()

if __name__ == "__main__":
    create_gui()
